In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
#from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import seaborn as sns

In [3]:
# Set display options to prevent truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

In [4]:
root_dir = './../data/complimentary_raw_data/raw_catalog_data/athena_pt_samples'
# List to hold all the file paths
file_paths = []

# Walk through the directory structure
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.csv') and not filename.startswith('.'):
            # Construct the full file path and add to the list
            file_paths.append(os.path.join(dirpath, filename))
print(f"Total CSV files found: {len(file_paths)}")

Total CSV files found: 166


In [5]:
combined_df = pd.read_csv('./../data/combined_data.csv')
combined_df.shape

(104944, 317)

In [6]:
categories = combined_df['product_type'].unique()

In [7]:
nan_df = combined_df.groupby('product_type').apply(lambda x: x.isna().sum())
nan_df.head(1)

,asin,active_ingredients,additional_product_information,age_range_description,allergen_information,asin.1,battery_capacity_unit,battery_capacity_value,battery_cell_composition,battery_weight_unit,battery_weight_value,binding,brand,browse_assignment,bullet_point,caffeine_content,capacity_unit,capacity_value,carbohydrate_sugars,carbohydrate_total,case_pack_quantity,chocolate_type,closure_type,coffee_roast_type,color,container_type,country_as_labeled,coverage,department,description,diet_type,directions,dosage_form,energy_content,energy_unit,fat_unit,finish_type,flavor,formulation_type,full_cure_time_unit,full_cure_time_value,gl_product_group_type,gtin,hair_type,handle_material,honey_type,identity_package_type,ingredients,is_electric,is_waterproof,item_diameter_unit,item_diameter_value,item_firmness_description,item_form,item_height,item_height_unit,item_length,item_length_unit,item_package_quantity,item_package_weight_unit,item_package_weight_value,item_shape,item_type_keyword,item_volume_unit,item_volume_value,item_width,item_width_unit,liquid_contents_description,liquid_volume_unit,liquid_volume_value,list_price_currency,list_price_value,manufacturer,material,material_feature,material_type_free,model_number,monounsaturated_fat,num_batteries_quantity,num_batteries_type,number_of_blades,number_of_items,number_of_pieces,occasion,operation_mode,package_size_name,package_type,package_type_name,parent_asin,part_number,pattern,pattern_type,plant_or_animal_product_type,ply,polyunsaturated_fat,power_source_type,primary_supplement_type,product_benefit,product_grade,product_type,protein_source,protein_unit,protein_value,recommended_uses_for_product,reusability,saturated_fat,scent,serving_description,serving_quantity,shelf_life_unit,shelf_life_value,shipping_weight_unit,shipping_weight_value,size,skin_tone,skin_type,snack_food_bar_type,sodium_unit,sodium_value,source_animal,special_feature,special_ingredients,specialty,specific_uses_for_product,style_name,sub_brand,subject_keyword,sun_protection_unit,sun_protection_value,surface_recommendation,target_audience,target_gender,target_species,tea_variety,thesaurus_attribute_keyword,thesaurus_subject_keyword,title,total_eaches,total_fat,trans_fat,unable_to_parse,unit_count,unit_count_type,upc,variety,voltage_unit,voltage_value,weight_unit,weight_value,winning_images,head_style,head_type,installation_type,gl_family,gl_name,gl_code,category_code,subcategory_code,glance_view_band,browse_node_ids,browse_node_names,edge_path_names,product_types,nail_polish_type,descriptionsource,antenna_description,compatible_devices,compatible_with_vehicle_type,connector_type,form_factor,frame_material,gem_type,headphones_form_factor,ink_color,keyboard_description,lamp_type,lens_type,light_source_type,light_type,mounting_type,number_of_speeds,occasion_type,seat_material_type,sport_type,stone_color,theme,top_material,arch_type,auto_part_position,back_material,band_material,base_type,bottom_style,bowl_material_type,chain_type,clasp_type,collar_style,collection,compatible_phone_models,construction_type,cup_size,exterior_finish,fill_material,fit_type,furniture_finish,hand_orientation,hardware_interface,headphones_jack,insole_type,leg_style,lining_description,metal_stamp,neck_style,number_of_shelves,number_of_wheels,orientation,outer_material,pearl_shape,platform,pocket_description,primary_stone_gem_type,rise_style,seasons,setting_type,shaft_height_unit,shaft_height_value,shelf_type,sleeve_type,stone_shape,surround_sound_channel_configuration,top_style,total_usb_ports,water_resistance_level,weave_type,wireless_provider,gv,si,style,alert_type,back_finding,care_instructions,closure_material,connectivity_protocol,connectivity_technology,control_method,control_type,controller_type,controls_type,deprecated_variation_theme_id,deprecated_variation_theme_name,drain_type,drive_system,engine_type,expected_blooming_period,expected_planting_period,external_testing_certification,fabric_type,fabric_wash,filter_type,fixture_type,focus_

In [8]:
nan_df["total_records"] = combined_df['product_type'].value_counts()

In [9]:
data = combined_df

In [10]:
data = data.select_dtypes(exclude=['float64', 'int'])

In [11]:
data.shape

(104944, 170)

In [12]:
data.to_csv('./../data/object_data.csv', index = False)

In [13]:
unique_df = data.groupby('product_type').apply(lambda x: x.nunique())

In [14]:
unique_df['total_records'] = nan_df['total_records']

In [15]:
unique_df.columns

Index(['asin', 'active_ingredients', 'additional_product_information',
       'age_range_description', 'allergen_information', 'asin.1',
       'battery_capacity_unit', 'battery_cell_composition',
       'battery_weight_unit', 'binding',
       ...
       'shaft_height_value', 'shelf_type', 'sleeve_type', 'stone_shape',
       'top_style', 'water_resistance_level', 'weave_type',
       'wireless_provider', 'snapshot_day', 'total_records'],
      dtype='object', length=171)

In [16]:
categories = unique_df.index
categories

Index(['BABY_BOTTLE', 'BAKING_MIX', 'BATHWATER_ADDITIVE', 'BATTERY',
       'BLOOD_PRESSURE_MONITOR', 'BODY_DEODORANT', 'BODY_LUBRICANT', 'BREAD',
       'BREAKFAST_CEREAL', 'CAKE', 'CANDLE', 'CANDY'],
      dtype='object', name='product_type')

In [17]:
unique_df

,asin,active_ingredients,additional_product_information,age_range_description,allergen_information,asin.1,battery_capacity_unit,battery_cell_composition,battery_weight_unit,binding,brand,browse_assignment,bullet_point,caffeine_content,capacity_unit,chocolate_type,closure_type,coffee_roast_type,color,container_type,country_as_labeled,coverage,department,description,diet_type,directions,dosage_form,energy_unit,fat_unit,finish_type,flavor,gl_product_group_type,hair_type,handle_material,identity_package_type,ingredients,is_electric,item_diameter_unit,item_firmness_description,item_form,item_height_unit,item_length_unit,item_package_weight_unit,item_shape,item_type_keyword,item_volume_unit,item_width_unit,liquid_contents_description,liquid_volume_unit,list_price_currency,manufacturer,material,material_feature,material_type_free,model_number,num_batteries_type,operation_mode,package_size_name,package_type,package_type_name,parent_asin,part_number,pattern,pattern_type,plant_or_animal_product_type,power_source_type,primary_supplement_type,product_benefit,product_type,protein_source,protein_unit,recommended_uses_for_product,reusability,scent,serving_description,shelf_life_unit,shipping_weight_unit,size,skin_tone,skin_type,sodium_unit,source_animal,special_feature,special_ingredients,specialty,specific_uses_for_product,sub_brand,subject_keyword,sun_protection_unit,sun_protection_value,surface_recommendation,target_audience,target_gender,target_species,tea_variety,thesaurus_attribute_keyword,thesaurus_subject_keyword,title,unit_count_type,upc,variety,voltage_unit,weight_unit,winning_images,installation_type,gl_family,browse_node_ids,browse_node_names,product_types,antenna_description,compatible_devices,compatible_with_vehicle_type,connector_type,form_factor,frame_material,gem_type,headphones_form_factor,ink_color,keyboard_description,lamp_type,light_source_type,light_type,mounting_type,occasion_type,sport_type,stone_color,theme,top_material,arch_type,auto_part_position,back_material,band_material,base_type,bottom_style,bowl_material_type,chain_type,clasp_type,collar_style,collection,compatible_phone_models,construction_type,cup_size,exterior_finish,fill_material,fit_type,hand_orientation,hardware_interface,headphones_jack,leg_style,lining_description,metal_stamp,neck_style,orientation,outer_material,pearl_shape,platform,pocket_description,primary_stone_gem_type,seasons,setting_type,shaft_height_unit,shaft_height_value,shelf_type,sleeve_type,stone_shape,top_style,water_resistance_level,weave_type,wireless_provider,snapshot_day,total_records
product_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BABY_BOTTLE,4296,0,25,57,6,4296,0,3,2,18,741,59,2504,0,8,0,5,0,386,0,21,1,52,2607,0,35,0,0,0,7,16,25,1,0,2,25,1,0,2,3,1,1,1,9,38,7,1,1,2,1,979,176,1,28,2312,3,0,5,0,3,716,3063,20,1,1,6,0,0,1,0,0,13,0,8,1,1,1,559,0,0,0,0,39,1,1,34,15,515,0,0,0,55,3,0,0,176,256,4185,23,2935,0,1,1,4220,2,3,30,30,28,0,2,0,2,7,2,0,0,0,0,0,1,1,0,7,11,0,16,0,0,0,0,0,0,0,0,0,0,0,10,2,0,1,1,0,2,0,1,0,0,1,1,0,0,5,0,0,0,0,12,0,0,0,0,0,0,0,1,0,0,1,5098
BAKING_MIX,9174,14,165,10,42,9174,0,0,0,14,1474,232,6045,3,1,0,0,0,180,32,27,0,11,6143,4,1036,0,3,2,1,2081,21,0,0,3,2803,1,2,2,26,1,1,1,2,222,5,1,2,4,1,1659,24,8,3,1184,0,0,0,0,8,999,2092,35,0,1,2,0,0,1,2,2,0,0,41,711,2,1,1863,0,0,2,0,15,15,33,1,6,696,0,0,0,8,3,0,0,489,135,9050,20,5004,7,0,1,8516,0,3,200,195,77,0,0,0,0,18,0,0,0,0,0,0,0,0,0,27,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,10000
BATHWATER_ADDITIVE,7385,9,16,14,5,7385,0,1,1,19,2050,261,4633,0,1,0,5,0,570,1,28,12,40,5244,2,717,0,2,1,9,210,28,14,2,1,1587,1,1,2,77,1,1,1,2,208,9,1,6,5,1,2152,102,9,25,1678,3,1,25,0,12,1185,4879,54,5,5,2,4,189,1,1,1,20,1,1584,4,2,1,1053,14,35,1,0,83,20,26,106,15,731,1,1,0,33,3,1,2,443,475,7249,29,4202,224,0,1,6385,1,3,192,183,112,0,2,1,0,1,0,1,0,0,0,0,0,1,0,4,1,0,5,

In [18]:
category_attribute = {}
threshold = .05
for index, row in unique_df.iterrows():
    total_records = row[-1]
    category_attribute[index] = {}
    for column_name, value in row.items():
        if value/total_records < threshold:
            category_attribute[index][column_name] ={}

In [19]:
unique_df.shape

(12, 171)

In [20]:
threshold = .5
attributes_set = set()
#drop columns with more than 50% unique values
def get_valid_attributes(row):
    valid_columns = []
    total_records = row['total_records']
    for col in unique_df.columns[:-1]:  # Exclude 'category_total_records'
        if row[col] / total_records < threshold:  # Check for less than 50% unique values
            valid_columns.append(col)
            attributes_set.add(col)
    return valid_columns

In [21]:
unique_df['attributes'] = unique_df.apply(get_valid_attributes, axis = 1)

In [22]:
unique_df.columns

Index(['asin', 'active_ingredients', 'additional_product_information',
       'age_range_description', 'allergen_information', 'asin.1',
       'battery_capacity_unit', 'battery_cell_composition',
       'battery_weight_unit', 'binding',
       ...
       'shelf_type', 'sleeve_type', 'stone_shape', 'top_style',
       'water_resistance_level', 'weave_type', 'wireless_provider',
       'snapshot_day', 'total_records', 'attributes'],
      dtype='object', length=172)

In [23]:
len(attributes_set)

166

In [24]:
data = data[[x for x in attributes_set]]

In [25]:
data.shape

(104944, 166)

In [26]:
nan_df = data.groupby('product_type').apply(lambda x: x.isna().sum())
nan_df

,browse_assignment,surface_recommendation,material_feature,bottom_style,coffee_roast_type,is_electric,product_type,primary_stone_gem_type,pattern,setting_type,scent,package_size_name,item_length_unit,fat_unit,shelf_type,chocolate_type,base_type,item_package_weight_unit,target_gender,stone_color,variety,pocket_description,lining_description,snapshot_day,stone_shape,capacity_unit,protein_source,tea_variety,frame_material,num_batteries_type,part_number,construction_type,operation_mode,energy_unit,collar_style,department,skin_tone,weight_unit,hardware_interface,headphones_form_factor,fit_type,light_type,exterior_finish,specialty,caffeine_content,flavor,compatible_devices,light_source_type,orientation,reusability,antenna_description,battery_cell_composition,leg_style,hand_orientation,pearl_shape,upc,liquid_volume_unit,seasons,manufacturer,special_ingredients,band_material,weave_type,top_style,directions,gl_product_group_type,product_types,container_type,protein_unit,serving_description,sun_protection_value,compatible_phone_models,battery_weight_unit,coverage,outer_material,pattern_type,additional_product_information,voltage_unit,package_type,material_type_free,material,mounting_type,color,handle_material,arch_type,sodium_unit,shaft_height_unit,hair_type,shaft_height_value,thesaurus_subject_keyword,binding,plant_or_animal_product_type,source_animal,clasp_type,size,bowl_material_type,fill_material,primary_supplement_type,sub_brand,subject_keyword,compatible_with_vehicle_type,sleeve_type,active_ingredients,back_material,specific_uses_for_product,installation_type,gem_type,metal_stamp,auto_part_position,neck_style,package_type_name,ink_color,diet_type,lamp_type,power_source_type,parent_asin,description,sun_protection_unit,cup_size,form_factor,headphones_jack,chain_type,occasion_type,item_height_unit,browse_node_ids,sport_type,age_range_description,identity_package_type,closure_type,product_benefit,platform,shelf_life_unit,allergen_information,list_price_currency,unit_count_type,water_resistance_level,target_audience,item_shape,special_feature,bullet_point,item_volume_unit,shipping_weight_unit,theme,recommended_uses_for_product,item_firmness_description,item_width_unit,brand,top_material,dosage_form,liquid_contents_description,country_as_labeled,thesaurus_attribute_keyword,model_number,item_type_keyword,item_form,connector_type,keyboard_description,skin_type,item_diameter_unit,gl_family,target_species,browse_node_names,finish_type,wireless_provider,ingredients,collection,battery_capacity_unit
product_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BABY_BOTTLE,0,5098,5097,5098,5098,4277,0,5098,5049,5098,5087,5086,2060,5098,5098,5098,5098,862,3933,5098,5098,5098,5095,0,5098,3523,5098,5098,5096,5094,1017,5098,5098,5098,5098,3692,5098,2357,5096,5098,5091,5097,5096,5096,5098,5061,5096,5096,5098,5098,5098,5095,5098,5098,5098,1491,4992,4907,233,5097,5098,5098,5098,5050,0,0,5098,5098,5097,5098,5095,5094,5096,5090,5097,5072,5092,5098,1916,3538,5098,1641,5098,5098,5098,5098,5096,5098,4458,1029,5096,5098,5098,2959,5098,5098,5098,5031,3332,5098,5098,5098,5098,5018,5094,5098,5097,5098,5098,5073,5098,5098,5098,5088,3086,586,5098,5095,5061,5098,5098,5069,2060,0,5020,888,5092,5082,5098,5098,5003,5089,3176,3214,5087,3275,5044,4996,758,4330,970,5036,5076,5093,2060,146,5098,5098,5096,4532,4315,2193,2,5091,5093,5098,5098,5098,0,5098,0,5080,5098,5045,5074,5098
BAKING_MIX,0,10000,9739,10000,10000,9995,0,10000,9925,9998,9940,10000,7394,7774,10000,10000,10000,2546,9823,10000,9991,10000,10000,0,10000,9999,9994,10000,10000,10000,6971,10000,10000,7730,10000,9944,10000,5791,10000,10000,9997,10000,10000,9481,9958,3169,10000,10000,10000,10000,10000,10000,10000,10000,10000,3504,9984,10000,1543,9957,10000,10000,10000,8315,0,0,6570,7686,8770,10000,9987,10000,10000,10000,10000,9822,10000,10000,9994,9948,10000,9534,10000,10000,9229,10000,10000,10000,

In [27]:
nan_df["total_records"] = combined_df['product_type'].value_counts()

In [28]:
threshold_nan = .9
attributes_set = set()
#drop columns with more than 50% unique values
def get_non_nan_attributes(row):
    valid_columns = []
    total_records = row['total_records']
    for col in nan_df.columns[:-1]:  # Exclude 'category_total_records'
        if row[col] / total_records < threshold:  # Check for less than 50% unique values
            valid_columns.append(col)
            attributes_set.add(col)
    return valid_columns

In [29]:
nan_df['attributes'] = nan_df.apply(get_non_nan_attributes, axis = 1)

In [30]:
len(attributes_set)

39

In [31]:
print(attributes_set)

{'weight_unit', 'voltage_unit', 'browse_assignment', 'material_type_free', 'item_height_unit', 'browse_node_ids', 'material', 'age_range_description', 'light_type', 'product_type', 'color', 'shelf_life_unit', 'flavor', 'thesaurus_subject_keyword', 'binding', 'scent', 'battery_cell_composition', 'unit_count_type', 'size', 'item_length_unit', 'bullet_point', 'upc', 'shipping_weight_unit', 'item_volume_unit', 'item_width_unit', 'manufacturer', 'brand', 'item_package_weight_unit', 'model_number', 'item_type_keyword', 'item_form', 'gl_family', 'gl_product_group_type', 'snapshot_day', 'product_types', 'browse_node_names', 'container_type', 'description', 'part_number'}


In [32]:
data = data[[x for x in attributes_set]]
data.shape

(104944, 39)

In [33]:
columns_to_drop = ['gl_product_group_type', 'browse_node_names', 'browse_assignment', 'weight_unit', 'item_type_keyword', 'item_height_unit', 
                   'part_number', 
                  'item_length_unit', 'item_width_unit', 'description', 'model_number', 'snapshot_day', 'size', 'shipping_weight_unit', 'product_types',
                  'item_form', 'item_package_weight_unit', 'thesaurus_subject_keyword', 'upc', 'browse_node_ids', 'gl_family', 'unit_count_type']
cleaned_df = data.drop(columns=columns_to_drop, errors='ignore')

In [34]:
cleaned_df.shape

(104944, 17)

In [35]:
attributes_list = cleaned_df.columns
attributes_list

Index(['voltage_unit', 'material_type_free', 'material',
       'age_range_description', 'light_type', 'product_type', 'color',
       'shelf_life_unit', 'flavor', 'binding', 'scent',
       'battery_cell_composition', 'bullet_point', 'item_volume_unit',
       'manufacturer', 'brand', 'container_type'],
      dtype='object')

In [36]:
cleaned_df.to_csv('./../data/cleaned__data.csv', index=False)

In [37]:
combined_df.shape

(104944, 317)

In [38]:
cleaned_df['color'].unique()

array(['Orange, Yellow, Gray', '["Multi"]', '["Black"]', ...,
       'Milk Chocolate Caramel', 'Blue, White, Red',
       'Blue (Pastel) Tower 1.5 lbs'], dtype=object)

In [39]:
import ast
def normalize_color_entry(entry):
    if pd.isna(entry):
        return ''
    
    # Convert all entries to string
    entry = str(entry)
    
    # Remove entries that start with a digit
    if entry and entry[0].isdigit():
        return ''
    
    # Try to convert JSON-like strings
    try:
        colors = ast.literal_eval(entry)
        if isinstance(colors, list):
            return ', '.join(colors)
    except (ValueError, SyntaxError):
        pass
    
    # Split by common separators
    if ',' in entry:
        colors = [color.strip() for color in entry.split(',')]
        return ', '.join(colors)
    
    return entry

In [40]:
cleaned_df['color'] = cleaned_df['color'].apply(normalize_color_entry)

In [41]:
# Remove non-color-related information
color_keywords = ['orange', 'yellow', 'gray', 'multi', 'black', 'blue', 'white', 'red', 'pastel', 'tower', 'lbs']

In [42]:
def filter_colors(entry):
    colors = [color for color in entry.split(', ') if any(keyword in color.lower() for keyword in color_keywords)]
    return ', '.join(colors)

cleaned_df['filtered_color'] = cleaned_df['color'].apply(filter_colors)


In [43]:
cleaned_df['color'] = cleaned_df['filtered_color']
cleaned_df = cleaned_df.drop(columns=['filtered_color'])

In [44]:
cleaned_df.to_csv('./../data/cleaned__data1.csv', index=False)

In [45]:
def normalize_entry(entry):
    if pd.isna(entry):
        return ''
    
    # Convert all entries to string
    entry = str(entry)
    
    # Remove entries that start with a digit
    if entry and entry[0].isdigit():
        return ''
    
    # Try to convert JSON-like strings
    try:
        parsed_entry = ast.literal_eval(entry)
        if isinstance(parsed_entry, list):
            return ', '.join(map(str, parsed_entry))
    except (ValueError, SyntaxError):
        pass
    
    # Split by common separators if needed (specific to the 'color' column in the example)
    if ',' in entry:
        parts = [part.strip() for part in entry.split(',')]
        return ', '.join(parts)
    
    return entry

# Apply the normalization function to all columns
for column in cleaned_df.columns:
    cleaned_df[column] = cleaned_df[column].apply(normalize_entry)

In [46]:
cleaned_df.shape

(104944, 17)

In [47]:
print(combined_df.columns)

Index(['asin', 'active_ingredients', 'additional_product_information',
       'age_range_description', 'allergen_information', 'asin.1',
       'battery_capacity_unit', 'battery_capacity_value',
       'battery_cell_composition', 'battery_weight_unit',
       ...
       'video_capture_resolution', 'watch_movement_type',
       'wireless_communication_technology', 'zoom_type', 'shoe_width_unit',
       'shoe_width_value', 'nut_seed_type', 'snapshot_day', 'region_id',
       'marketplace_id'],
      dtype='object', length=317)


In [48]:
df = combined_df.select_dtypes(['float64', 'int'])

In [49]:
for col in df.columns:
    print(col, df[col].nunique())

battery_capacity_value 77
battery_weight_value 272
capacity_value 125
carbohydrate_sugars 151
carbohydrate_total 184
case_pack_quantity 46
energy_content 285
formulation_type 0
full_cure_time_unit 0
full_cure_time_value 0
gtin 8972
honey_type 0
is_waterproof 0
item_diameter_value 29
item_height 2289
item_length 2335
item_package_quantity 125
item_package_weight_value 5790
item_volume_value 579
item_width 2215
liquid_volume_value 307
list_price_value 3817
monounsaturated_fat 18
num_batteries_quantity 35
number_of_blades 3
number_of_items 90
number_of_pieces 44
occasion 0
ply 0
polyunsaturated_fat 19
product_grade 0
protein_value 83
saturated_fat 172
serving_quantity 159
shelf_life_value 816
shipping_weight_value 5548
snack_food_bar_type 0
sodium_value 226
style_name 0
total_eaches 55
total_fat 133
trans_fat 24
unable_to_parse 0
unit_count 2332
voltage_value 72
weight_value 3309
head_style 0
head_type 0
gl_name 0
gl_code 40
category_code 0
subcategory_code 0
glance_view_band 0
edge_path_

In [50]:
selected_df=df[['item_height', 'item_length', 'item_volume_value', 'item_width', 'battery_capacity_value', 'battery_weight_value']]

In [51]:
cleaned_df = cleaned_df.join(selected_df)

In [52]:
cleaned_df.shape

(104944, 23)

In [53]:
cleaned_df.to_csv('./../data/cleaned_data.csv', index = False)

In [54]:
cleaned_df.shape

(104944, 23)

In [55]:
# Define the columns to bin
columns_to_bin = [
    'item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value'
]

In [56]:
filtered_df = cleaned_df[['product_type','item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value']]

In [57]:
filtered_df.head(5)

,product_type,item_height,item_length,item_volume_value,item_width,battery_capacity_value,battery_weight_value
0,BABY_BOTTLE,NaN,NaN,NaN,NaN,NaN,NaN
1,BABY_BOTTLE,NaN,NaN,NaN,NaN,NaN,NaN
2,BABY_BOTTLE,9.50,11.00,NaN,4.00,NaN,NaN
3,BABY_BOTTLE,2.38,7.00,NaN,2.38,NaN,NaN
4,BABY_BOTTLE,2.55,10.05,NaN,4.75,NaN,NaN


In [58]:
columns_to_aggregate = [
    'item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value'
]
agg_df = filtered_df.groupby('product_type')[columns_to_aggregate].agg(['min', 'max', 'count'])

In [59]:
agg_df

item_height                    item_length                    item_volume_value                    item_width                    battery_capacity_value                battery_weight_value               
                               min          max count         min          max count               min          max count        min          max count                    min      max count                  min      max count
product_type                                                                                                                                                                                                                     
BABY_BOTTLE                    0.0   235.000000  3038         0.0   201.000000  3038             0.000  3030000.000   768        0.0   201.000000  3038                    NaN      NaN     0                 0.00     5.00     4
BAKING_MIX                     0.0   448.818897  2606         0.0   413.385826  2606             0.000     1470.000   152        0.0   295.275590  2606                    NaN      NaN     0                  NaN      NaN     0
BATHWATER_ADDITIVE             0.0    78.740157  3770         0.0    70.870000  3770             0.020  6188000.000  1038        0.0    95.000000  3770                    NaN      NaN     0                 5.40   200.00     5
BATTERY                        0.0    95.275590  3699         0.0  2685.000000  3699             0.000     4588.000    98        0.0  1976.771652  3699                    1.0  90000.0  3343                 0.00  1200.00  1033
BLOOD_PRESSURE_MONITOR         0.0   196.850000  1088         0.0   196.850000  1088             0.140   464400.000    40        0.0   161.000000  1088                    NaN      NaN     0                 0.00   350.00   114
BODY_DEODORANT                 0.0   190.000000  4241         0.0    45.000000  4241             0.030     2857.000  6593        0.0   220.500000  4241                    NaN      NaN     0                 0.00     5.00    78
BODY_LUBRICANT                 0.0   590.550000  3624         0.0   590.550000  3624             0.100     3800.000  2841        0.0   590.550000  3624                 4400.0   4400.0     1                 0.00    20.00    16
BREAD                          0.0    96.456693  2395         0.0    24.409449  2395             0.061      226.800    37        0.0    96.456693  2395                 4400.0   4400.0     4                  NaN      NaN     0
BREAKFAST_CEREAL               0.0   826.770000  3581         0.0   433.070000  3581             0.350      583.000    96        0.0   244.094488  3581                 8800.0   8800.0     3                  NaN      NaN     0
CAKE                           0.0  1496.060000  1832         0.0  9527.540000  1832             0.000    57420.000   191        0.0  4566.920000  1832                    NaN      NaN     0               454.00   454.00     1
CANDLE                         0.0    72.000000  4665         0.0    77.550000  4665             0.000    11496.816   184        0.0    72.000000  4665                    NaN      NaN     0                 0.10   130.00    34
CANDY                          0.0  1035.431000  2636         0.0   456.692913  2636             0.350     1000.000    37        0.0   374.015000  2636                    NaN      NaN     0                18.34    18.34     1

In [60]:
# Define the columns to bin
columns_to_bin = [
    'item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value'
]

def bin_columns(group, num_bins=10):
    product_type = group['product_type'].iloc[0]  # Extract product_type
    binned_data = group.copy()
    
    for column in columns_to_bin:
        if group[column].notnull().sum() > 0:  # Check if there are non-null values
            binned_data[column + '_bin'] = pd.cut(
                group[column], 
                bins=num_bins, 
                labels=[f'{product_type}_{i+1}' for i in range(num_bins)],
                duplicates='drop'
            )
            # Add an empty string to the categories to allow replacement
            binned_data[column + '_bin'] = binned_data[column + '_bin'].cat.add_categories([''])
        else:
            binned_data[column + '_bin'] = np.nan
    return binned_data

# Apply the function to each group
binned_df = filtered_df.groupby('product_type').apply(bin_columns).reset_index(drop=True)

# Replace NaN values with empty strings
binned_df = binned_df.fillna('')

binned_df.head(5)

,product_type,item_height,item_length,item_volume_value,item_width,battery_capacity_value,battery_weight_value,item_height_bin,item_length_bin,item_volume_value_bin,item_width_bin,battery_capacity_value_bin,battery_weight_value_bin
0,BABY_BOTTLE,,,,,,,,,,,,
1,BABY_BOTTLE,,,,,,,,,,,,
2,BABY_BOTTLE,9.5,11.0,,4.0,,,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,
3,BABY_BOTTLE,2.38,7.0,,2.38,,,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,
4,BABY_BOTTLE,2.55,10.05,,4.75,,,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,


In [61]:
cleaned_df.columns

Index(['voltage_unit', 'material_type_free', 'material',
       'age_range_description', 'light_type', 'product_type', 'color',
       'shelf_life_unit', 'flavor', 'binding', 'scent',
       'battery_cell_composition', 'bullet_point', 'item_volume_unit',
       'manufacturer', 'brand', 'container_type', 'item_height', 'item_length',
       'item_volume_value', 'item_width', 'battery_capacity_value',
       'battery_weight_value'],
      dtype='object')

In [62]:
binned_df = binned_df.drop(['item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value', 'product_type'],axis = 1)

In [63]:
binned_df.shape

(104944, 6)

In [64]:
merged_df = pd.merge(cleaned_df, binned_df, left_index=True, right_index=True, suffixes=('', '_bin'))

In [65]:
merged_df.shape

(104944, 29)

In [66]:
merged_df.head(5)

,voltage_unit,material_type_free,material,age_range_description,light_type,product_type,color,shelf_life_unit,flavor,binding,scent,battery_cell_composition,bullet_point,item_volume_unit,manufacturer,brand,container_type,item_height,item_length,item_volume_value,item_width,battery_capacity_value,battery_weight_value,item_height_bin,item_length_bin,item_volume_value_bin,item_width_bin,battery_capacity_value_bin,battery_weight_value_bin
0,,BPA Free,,Baby,,BABY_BOTTLE,"Orange, Yellow, Gray",,,miscellaneous,,,"[Durable, single-wall, BPA-free Tritan construction. Set of 3., Easy to fill twist-body allows you to add ice cubes, lemons or limes, Spill-resistant sealing lid; textured, non-slip sleeve, Holds 16.9-ounces; will fit standard cup holders, Dishwasher safe; hand washing recommended]",,Lifetime Brands,Copco,,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,
1,,,,Kid,,BABY_BOTTLE,Multi,,,baby_product,,,,,Nuby,Nuby,,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,
2,,Phthalates Free,,,,BABY_BOTTLE,Black,,,baby_product,,,"[Insulated bottle cooler holds up to (4) standard 8oz. bottles, (3) wide-body or tall 8-9oz. bottles, (4) standard 4-5oz. newborn bottles or (3) wide-body 4-5oz. newborn bottles., Fully insulated and leak-proof with easy-to-clean lining and with snap closure; includes reusable ice pack, Slim-design is compact to fit in diaper bag, backpack, purse or travel bag; Exterior pocket plus snap handle to attach to stroller bar, Keep bottle and cotainers warm or cold with this PVC, lead and phthalate free case, Thoughtfully designed and manufactured by J.L. Childress - delivering high-quality products that are practical and multi-functional for over 30 years]",,J L Childress,J.L. Childress,,9.50,11.00,NaN,4.00,NaN,NaN,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,
3,,BPA Free,,Kid,,BABY_BOTTLE,multicolor,,,sports,,,"[Officially Licensed Collegiate Product, For newborns and up, Boldly decorated with colorful team graphics]",,Dreme Corp,Baby Fanatic,,2.38,7.00,NaN,2.38,NaN,NaN,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,
4,,BPA Free,BPA Free,Baby,,BABY_BOTTLE,,,,baby_product,,,"[Innovative Dr. Brown's Natural Flow Bottle, Patented, 2-piece internal vent system, Reduces feeding problems including colic, spit-up, burping and gas, Creates positive-pressure feeding that closely resembles breastfeeding, Prevents vacuum pressure and air bubbles]",,HANDI-CRAFT COMPANY,Dr. Brown's,,2.55,10.05,NaN,4.75,NaN,NaN,BABY_BOTTLE_1,BABY_BOTTLE_1,,BABY_BOTTLE_1,,


In [67]:
merged_df = merged_df.drop(['item_height', 'item_length', 'item_volume_value',
    'item_width', 'battery_capacity_value', 'battery_weight_value'],axis = 1)

In [68]:
merged_df.shape

(104944, 23)

In [69]:
merged_df.to_csv('./../data/final_data.csv', index = False)

In [70]:
category_attribute_frequency = {}
for index, row in merged_df.iterrows():
    category = row['product_type']
    if category not in category_attribute_frequency.keys():
        category_attribute_frequency[category] = {}
    else:
        for col in merged_df.columns:
            if col != 'product_type':
                if col not in category_attribute_frequency[category].keys():
                    category_attribute_frequency[category][col] = {}
                else:
                    value = str(row[col]).lower()
                    if value == '':
                        continue
                    if value not in category_attribute_frequency[category][col].keys():
                        category_attribute_frequency[category][col][value] = 1
                    else:
                        category_attribute_frequency[category][col][value] += 1

In [71]:
with open('./../data/category_attribute_frequency.json', 'w') as json_file:
    json.dump(category_attribute_frequency, json_file, indent=4)